In [ ]:
#@title Install Dependencies

# general
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine


# OOD Detection
!pip install pytorch-ood

try:
  import timm
except:
  !pip install timm
  import timm

try:
  import cleanlab
except:
  !pip install cleanlab

## Ensemble-Specific
!pip install Typing

  Using cached torch-1.10.0-cp37-cp37m-manylinux1_x86_64.whl (881.9 MB)
  Using cached torchvision-0.11.1-cp37-cp37m-manylinux1_x86_64.whl (23.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0
    Uninstalling torch-1.12.0:
      Successfully uninstalled torch-1.12.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.0
    Uninstalling torchvision-0.13.0:
      Successfully uninstalled torchvision-0.13.0


In [ ]:
#@title Packages

## General
import os
from pathlib import Path
import pickle

## Data Analysis 
import pandas as pd
import numpy as np

## Torch Specific
# !pip install torchvision
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.transforms as T

## OOD Specific
# cleanlab
import cleanlab
from cleanlab.outlier import OutOfDistribution
from cleanlab.rank import find_top_issues
import gc

In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
!pip install --upgrade torchvision

In [ ]:
#@title Import Data & Format Accordingly


train_path = '/content/train'
test_path = '/content/test'


train_dog = train_path + '/dog'
train_bird = train_path + '/bird'
train_rep = train_path + '/reptile' 

test_dog = test_path + '/dog'
test_bird = test_path + '/bird'
test_rep = test_path + '/reptile' 

batch_size = 1

# Get a set of pretrained model weights
weights = torchvision.models.ResNet50_Weights.DEFAULT.DEFAULT # .DEFAULT = best available weights from pretraining on ImageNet

# Get the transforms used to create our pretrained weights
auto_transforms = weights.transforms()

# Create training and testing DataLoaders as well as get a list of class names
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_path,
                                                                               test_dir=test_path,
                                                                               transform=auto_transforms, # perform same data transforms on our own data as the pretrained model
                                                                               batch_size = batch_size) # set mini-batch size to 32

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7f978feb53d0>,
 ['bird', 'dog', 'reptile'])

### Ensemble Model

#### helper functions

In [ ]:
def test_model(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device):
    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
    """
    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc
  


####Instantiate and test model

In [ ]:
class EnsembleModelTwo(nn.Module):   
  '''
  ensemble class will take two models this time and use their input to classify
  the superclass by learning the right weights between the two models
  in our case, this wil be a ResNet50 and Vit.

  So each incoming feature vector will be a 3x1, and we will have two of them.
  The output will be a 3x1 
  softmax vector of the super classes 
  '''

  def __init__(self, modelA, modelB):
    super().__init__()
    self.modelA = modelA
    self.modelB = modelB
    self.classifier = nn.Linear(2 * 3, 3)
        
  def forward(self, x):
    x1 = self.modelA(x)
    x2 = self.modelB(x)
    x = torch.cat((x1, x2), dim=1)
    out = self.classifier(x)
    return out

In [ ]:
#instantiate large pre trained models and load our trained weights 
#initialize ensemble modles 
weights_resnet = torchvision.models.ResNet50_Weights.DEFAULT.DEFAULT
resnet_model = torchvision.models.resnet50(weights=weights_resnet)
in_ftrs = resnet_model.fc.in_features
out_fts = 3
resnet_model.fc = nn.Linear(in_ftrs, out_fts)
resnet_model.load_state_dict(torch.load('/content/resnet_50.pth')) #whatever path holds the model ...this is what it was on my VM 
resnet_model.eval()

resnet_model.to(device)


weights_vit = torchvision.models.ViT_L_16_Weights.IMAGENET1K_SWAG_E2E_V1.DEFAULT # .DEFAULT = best available weights from pretraining on ImageNet

vit_model = torchvision.models.vit_l_16(weights=weights_vit)
vit_model.heads = torch.nn.Sequential( 
    torch.nn.Linear(in_features=1024, 
                    out_features=3,
                    bias=True))
vit_model.heads.load_state_dict(torch.load('/content/vit_l_16_classifier.pth'))#whatever path holds the model ...this is what it was on my VM 
vit_model.eval()

vit_model.to(device)

In [ ]:
#instantiate enesemble in eval mode
ensemble_model_2 = EnsembleModelTwo(resnet_model, vit_model)


PATH = '/content/ensemble__2_best.pth'



# load in MY weights
ensemble_model_2.classifier.load_state_dict(torch.load(PATH))
ensemble_model_2.eval()


In [ ]:
#test ensemble 
# switch to cuda
if torch.cuda.is_available():
    ensemble_model_2.cuda()
  

# test_model(ensemble_model_2, test_dataloader, nn.CrossEntropyLoss(), device)


In [ ]:
#@title OOD Helpers

# Generates 2048-dimensional feature embeddings from images
def embed_images(model, dataloader):
    feature_embeddings = []
    for data in dataloader:
        images, labels = data
        with torch.no_grad():
            embeddings = model(images)
            feature_embeddings.extend(embeddings.numpy())
    feature_embeddings = np.array(feature_embeddings)
    return feature_embeddings  # each row corresponds to embedding of a different image

# Transform a single image
def embed_single_image(model, image):
    feature_embeddings = []
    # image, labels = data
    with torch.no_grad():
        embeddings = model(image)
        feature_embeddings.extend(embeddings.detach().cpu().numpy())
    feature_embeddings = np.array(feature_embeddings) # had to comment out here, since we switch to GPU 
    return feature_embeddings

In [ ]:
#@title OOD Instantiation

# Load pretrained neural network
ood_model = timm.create_model('resnet50', pretrained=True, num_classes=0)  # this is a pytorch network
ood_model.eval()  # eval mode disables training-time operators (like batch normalization)
if torch.cuda.is_available():
    ood_model.cuda()

# manual data transforms
manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Reshape all images to 224x224 (though some models may require different sizes)
    transforms.ToTensor(), # 2. Turn image values to between 0 & 1 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # 3. A mean of [0.485, 0.456, 0.406] (across each colour channel)
                         std=[0.229, 0.224, 0.225]) # 4. A standard deviation of [0.229, 0.224, 0.225] (across each colour channel),
])

#Create dataloaders for each superclass 
train_dog_dl, test_dog_dl, dog_classes = data_setup.create_dataloaders(
                                                            train_dir=train_dog,
                                                            test_dir=test_dog,
                                                            transform=manual_transforms,#use manual transforms so its the same in ensemble
                                                            batch_size=1) 

train_bird_dl, test_bird_dl, bird_classes = data_setup.create_dataloaders(
                                                            train_dir=train_bird,
                                                            test_dir=test_bird,
                                                            transform=manual_transforms,#use manual transforms so its the same in ensemble
                                                            batch_size=1) 

train_rep_dl, test_rep_dl, rep_classes = data_setup.create_dataloaders(
                                                            train_dir=train_rep,
                                                            test_dir=test_rep,
                                                            transform=manual_transforms,#use manual transforms so its the same in ensemble
                                                            batch_size=1) 



# generate ood predictions for every image in testing example

## gather training embeddings for each super class
train_feature_embeddings_dog = embed_images(ood_model, train_dog_dl)
train_feature_embeddings_bird = embed_images(ood_model, train_bird_dl)
train_feature_embeddings_rep = embed_images(ood_model, train_rep_dl)

## fit scores to training for each super class embeddings
# dog
dog_ood = OutOfDistribution()
train_ood_features_scores_dog = dog_ood.fit_score(features=train_feature_embeddings_dog)
# bird
bird_ood = OutOfDistribution()
train_ood_features_scores_bird = bird_ood.fit_score(features=train_feature_embeddings_bird)
# reptile
rep_ood = OutOfDistribution()
train_ood_features_scores_rep = rep_ood.fit_score(features=train_feature_embeddings_rep)


# delete to clear up RAM
del train_feature_embeddings_dog, train_feature_embeddings_bird, train_feature_embeddings_rep
gc.collect()

In [ ]:
#@title SubClass Model Instantiation

batch_size = 1

# Get a set of pretrained model weights
weights = torchvision.models.ResNet50_Weights.DEFAULT.DEFAULT # .DEFAULT = best available weights from pretraining on ImageNet

# Get the transforms used to create our pretrained weights
auto_transforms = weights.transforms()

#Create dataloaders for each superclass 
resnet_train_dog_dl, resnet_test_dog_dl, resnet_dog_classes = data_setup.create_dataloaders(
                                                                        train_dir=train_dog,
                                                                        test_dir=test_dog,
                                                                        transform=manual_transforms,#use manual transforms so its the same in ensemble
                                                                        batch_size=batch_size) 

resnet_train_bird_dl, resnet_test_bird_dl, resnet_bird_classes = data_setup.create_dataloaders(
                                                                        train_dir=train_bird,
                                                                        test_dir=test_bird,
                                                                        transform=manual_transforms,#use manual transforms so its the same in ensemble
                                                                        batch_size=batch_size) 

resnet_train_rep_dl, resnet_test_rep_dl, resnet_rep_classes = data_setup.create_dataloaders(
                                                                        train_dir=train_rep,
                                                                        test_dir=test_rep,
                                                                        transform=manual_transforms,#use manual transforms so its the same in ensemble
                                                                        batch_size=batch_size)

### DOG MODEL ###
# Instantiate Dog Model
dog_model = torchvision.models.resnet50()


# configure output
in_ftrs = dog_model.fc.in_features
out_fts = len(resnet_dog_classes)
dog_model.fc = nn.Linear(in_ftrs, out_fts)

# load in MY weights
dog_model.load_state_dict(torch.load('/content/DOG_resnet.pth'))
dog_model.eval()

# switch to cuda
if torch.cuda.is_available():
    dog_model.cuda()


### BIRD MODEL ###
# Instantiate BIRD Model
bird_model = torchvision.models.resnet50()

# configure output
in_ftrs = bird_model.fc.in_features
out_fts = len(resnet_bird_classes)
bird_model.fc = nn.Linear(in_ftrs, out_fts)

# load in MY weights
bird_model.load_state_dict(torch.load('/content/BIRD_resnet.pth'))
bird_model.eval()

# switch to cuda
if torch.cuda.is_available():
    bird_model.cuda()


### REPTILE MODEL ###
# Instantiate BIRD Model
rep_model = torchvision.models.resnet50()

# configure output
in_ftrs = rep_model.fc.in_features
out_fts = len(resnet_rep_classes)
rep_model.fc = nn.Linear(in_ftrs, out_fts)

# load in MY weights
rep_model.load_state_dict(torch.load('/content/REP_resnet.pth'))
rep_model.eval()

# switch to cuda
if torch.cuda.is_available():
    rep_model.cuda()



In [ ]:
# load in data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/neuralnet_competition_data-repository:latest')

# Extract images
# !unzip "neuralnet_competition_data/X_train.zip"
# !unzip "neuralnet_competition_data/X_test.zip" 

# LOAD-IN Competition Data !!!

# Create ordered list of filepaths 
test_filepaths = [('/content/test_shuffle/' + str(i) + '.jpg') for i in range(0, 9127)]


def preprocessor(image_filepath, transform):
        image = torchvision.io.read_image(image_filepath)
        transformed = transform(image)
        X = torch.reshape(transformed, (1, 3, 224, 224))
        return X.to(device).float()


# pre-process Testing Data
X_test = [preprocessor(x, T.Resize((224,224))) for x in test_filepaths]


In [ ]:
#@title Put it all together!

'''
First, it would most likely be beneficial to have batch_size = 1, so we basically loop thru one example at a time,
classify it as [bird, reptile, dog], check if sample is likely to be OOD, and if not, feed into pre-specified subclassifier
'''


# instantiate ensemble mode
model = ensemble_model_2

# threshold for OOD
threshold = 0.5 # up for discussion obv

# loop thru and predict
superclasses = []
subclasses = []
with torch.inference_mode():
    for image in X_test:

        # calculate outputs by running images through the network
        outputs = model(image)

        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        label = train_dataloader.dataset.classes[predicted] # this var is [dog, reptile, bird]
        superclasses.append(label)

        #create the embedding for the test image
        test_feature_embeddings = embed_single_image(ood_model, image)

        #based on the label (dog, bird, rep), we need use the corresponding embeddings to create a feature scores
        if label == 'bird': #MAKE SURE TEST_OOD_FEATURES_SCORE IS A NUMPY ARRAY WITH ONE ELEMENT
            test_ood_features_score = bird_ood.score(features=test_feature_embeddings)
        elif label == 'dog':
            test_ood_features_score = dog_ood.score(features=test_feature_embeddings)
        else:
            test_ood_features_score = rep_ood.score(features=test_feature_embeddings)

        # yup, test_odd_features_score is an array
        if test_ood_features_score[0] < threshold:
            subclass = 'novel'
        else:
            if label == 'dog':
                pred = dog_model(image)
                _, predicted = torch.max(pred.data, 1)
                subclass = resnet_test_dog_dl.dataset.classes[predicted]

            elif label == 'reptile':
                pred = rep_model(image)
                _, predicted = torch.max(pred.data, 1)
                subclass = resnet_test_rep_dl.dataset.classes[predicted]

            else:
                pred = bird_model(image)
                _, predicted = torch.max(pred.data, 1)
                subclass = resnet_test_bird_dl.dataset.classes[predicted]
        # append
        subclasses.append(subclass)


In [ ]:
def create_preds(superclasses, subclasses):
    super_preds, sub_preds = pd.read_csv('/content/predictions_sample.csv'), pd.read_csv('/content/predictions_sample.csv')
    super_preds['predictions'] = superclasses
    sub_preds['predictions'] = subclasses
    return super_preds, sub_preds


# convert & export
super_preds, sub_preds = create_preds(superclasses, subclasses)
super_preds.to_csv('/content/SUPER_preds.csv')
sub_preds.to_csv('/content/SUB_preds.csv')

In [ ]:
# Finally, SUBMIT !!
import aimodelshare as ai
from aimodelshare.aws import set_credentials


# Superclass Submission
SUPER_apiurl = "https://8vhobca1n7.execute-api.us-east-1.amazonaws.com/prod/m"
set_credentials(apiurl=SUPER_apiurl)

#Instantiate Competition
mycompetition= ai.Competition(SUPER_apiurl)

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = None,
                                 preprocessor_filepath=None,
                                 prediction_submission=superclasses)


AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.
Insert search tags to help users find your model (optional): ResNet/ViT Ensemble
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 193

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2651


In [ ]:
# Subclass Submission

SUB_apiurl= "https://arj1w1ffm6.execute-api.us-east-1.amazonaws.com/prod/m"
set_credentials(apiurl=SUB_apiurl)

#Instantiate Competition
mycompetition= ai.Competition(SUB_apiurl)

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = None,
                                 preprocessor_filepath=None,
                                 prediction_submission=subclasses)